# a notebook to clean the data before putting it into the vectorstore

In [288]:
import pandas as pd
import re

In [289]:
data = pd.read_csv('/Users/anthony/Code/tn_slackbot/data/messages2024-04-03-2025-04-10.csv')
# data = pd.read_csv('/Users/anthony/Code/tn_slackbot/data/messages2024-03-24-2025-03-24.csv')

In [290]:
data.columns

Index(['User', 'Message', 'Channel ID', 'Thread ID', 'Timestamp',
       'Raw Timestamp'],
      dtype='object')

In [291]:
data.head()

,User,Message,Channel ID,Thread ID,Timestamp,Raw Timestamp
0,U01TUBC73V0,<@U01TUBC73V0> has left the channel,C4WTA9ZBJ,NaN,2025-04-09T16:29:10-04:00,1.744231e+09
1,U03B5RBRVPU,"Hey team, is there new password requirements o...",C4WTA9ZBJ,1.744231e+09,2025-04-09T16:28:30-04:00,1.744231e+09
2,U030N287QSZ,<https://textnow.atlassian.net/browse/TS-3664>,C4WTA9ZBJ,1.744231e+09,2025-04-09T16:30:15-04:00,1.744231e+09
3,U03B5RBRVPU,<!subteam^S0390SUMXKN>,C4WTA9ZBJ,1.744231e+09,2025-04-09T16:30:53-04:00,1.744231e+09
4,U086D2Q3M6G,Oh yes there are,C4WTA9ZBJ,1.744231e+09,2025-04-09T16:41:20-04:00,1.744231e+09


In [292]:
data.isna().sum()

User              47
Message          266
Channel ID         0
Thread ID        934
Timestamp          0
Raw Timestamp      0
dtype: int64

In [293]:
data = data.dropna()

In [294]:
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

/var/folders/93/k3ns58nj0bl_wbr16m4pzzn00000gn/T/ipykernel_49132/1376026118.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data["Timestamp"] = pd.to_datetime(data["Timestamp"])


In [295]:
print(df.iloc[0])

Channel ID                                           [Channel 45]
Thread ID                                       1714579526.341119
conversation    Nicole: Hey @Vincent, we are migrating the cal...
Name: 0, dtype: object


In [296]:
from random_names import get_random_names
random_names = get_random_names()
len(random_names)

800

In [297]:
import html
import re

user_map = {
    
}

channel_map = {

}

user_group_map = {

}

def replace_channel(match): 
    channel_id = match.group(1)
    if channel_id in channel_map:
        return channel_map[channel_id]
    else:
        name = f"[Channel {len(channel_map ) + 1}]"
        channel_map[channel_id] = name
        return name

def replace_mention(match):
    user_id = match.group(1)
    if user_id in user_map:
        return f"@{user_map[user_id]}"
    else:
        name = random_names.pop()
        user_map[user_id] = name
        return f"@{name}"
    
def replace_user_group(match):
    user_group = match.group(1)
    if user_group in user_group_map:
        return f"{user_group_map[user_group]}"
    else:
        name = f"@[User Group {len(user_group_map) + 1}]"
        user_group_map[user_group] = name
        return name


def clean_message(text):
    # Unescape HTML
    text = html.unescape(text)

    # Trim whitespace
    text = text.strip()

    # sub user mentions
    text = re.sub(r"<@([A-Z0-9]+)>", replace_mention, text)
    
    # sub links
    text = re.sub(r"<(https?://[^>]+)>", "[URL]", text)

    # sub emails
    text = re.sub(r"\S+@\S+", "[EMAIL]", text)

    # sub channels
    text = re.sub(r"<#(C[0-9A-Z]+)(?:\|[^>]*)?>", replace_channel, text)

    # code blocks
    text = re.sub(r"```(.*?)```", r"\1", text, flags=re.DOTALL)
    text = re.sub(r"`(.*?)`", r"\1", text)

    # system messages
    text = re.sub(r"<![a-z]+>", "[system:channel-notification]", text)

    # groups
    text = re.sub(r"<!subteam\^([A-Z0-9]+)>", replace_user_group, text)

    # **Handle emphasis formatting**
    text = re.sub(r"\*(.*?)\*", r"\1", text)  # *bold*
    text = re.sub(r"_(.*?)_", r"\1", text)    # _italic_
    text = re.sub(r"~(.*?)~", r"\1", text)    # ~strikethrough~

    return text

data['cleaned_message'] = data['Message'].apply(clean_message)
    


In [298]:
data.head(1)
data.iloc[0]['Thread ID']

np.float64(1744230510.076049)

In [299]:
data.groupby(by=['Thread ID']).count()

,User,Message,Channel ID,Timestamp,Raw Timestamp,cleaned_message
Thread ID,,,,,,
1.710960e+09,1,1,1,1,1,1
1.712233e+09,96,96,96,96,96,96
1.712243e+09,6,6,6,6,6,6
1.712252e+09,47,47,47,47,47,47
1.712254e+09,2,2,2,2,2,2
...,...,...,...,...,...,...
1.744215e+09,14,14,14,14,14,14
1.744225e+09,37,37,37,37,37,37
1.744228e+09,3,3,3,3,3,3


In [300]:
# custom groupby method
def format_conversation(group):
    # sort by timestamp
    messages = []
    group = group.sort_values(by='Timestamp', ascending=True)
    
    for _, row in group.iterrows():
        user_id = row['User']
        if user_id not in user_map:
            user_map[user_id] = random_names.pop()
        user = user_map[user_id]
        messages.append(f"{user}: {row['cleaned_message']}")
    return '\n'.join(messages)

# group the data by timestamp and thread id 
df = data.groupby(by=['Channel ID', 'Thread ID']).apply(format_conversation).reset_index(name='conversation')

/var/folders/93/k3ns58nj0bl_wbr16m4pzzn00000gn/T/ipykernel_49132/2623178627.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = data.groupby(by=['Channel ID', 'Thread ID']).apply(format_conversation).reset_index(name='conversation')


In [301]:
df

,Channel ID,Thread ID,conversation
0,C0196L2UC3G,1.714580e+09,"Collin: Hey @Calvin, we are migrating the call..."
1,C0196L2UC3G,1.714760e+09,Angel: Hey team - just forwarding an email fro...
2,C0196L2UC3G,1.714832e+09,"Mia: anyone else (weekend, i know) having VPN ..."
3,C0196L2UC3G,1.718988e+09,Chad: Good afternoon! @Collin Do you know if t...
4,C0196L2UC3G,1.720460e+09,"Marco: Hey all, got an ask from Lokesh and was..."
...,...,...,...
1168,CFBF2NKRS,1.744142e+09,"Dominick: Hi @[User Group 6], we created a tic..."
1169,CFBF2NKRS,1.744142e+09,Saul: :alert: We're aware of some issues with ...
1170,CFBF2NKRS,1.744144e+09,Taryn: Hey @[User Group 6] looks like github a...
1171,CFBF2NKRS,1.744228e+09,Johnathan: Hey @[User Group 6]\n\nI have a pro...


In [302]:
def get_channel_name(channel_id):
    if channel_id in channel_map:
        return channel_map[channel_id]
    else:
        name = f"[Channel {len(channel_map ) + 1}]"
        channel_map[channel_id] = name
        return name

df['Channel ID'] = df['Channel ID'].apply(get_channel_name)
df


,Channel ID,Thread ID,conversation
0,[Channel 45],1.714580e+09,"Collin: Hey @Calvin, we are migrating the call..."
1,[Channel 45],1.714760e+09,Angel: Hey team - just forwarding an email fro...
2,[Channel 45],1.714832e+09,"Mia: anyone else (weekend, i know) having VPN ..."
3,[Channel 45],1.718988e+09,Chad: Good afternoon! @Collin Do you know if t...
4,[Channel 45],1.720460e+09,"Marco: Hey all, got an ask from Lokesh and was..."
...,...,...,...
1168,[Channel 34],1.744142e+09,"Dominick: Hi @[User Group 6], we created a tic..."
1169,[Channel 34],1.744142e+09,Saul: :alert: We're aware of some issues with ...
1170,[Channel 34],1.744144e+09,Taryn: Hey @[User Group 6] looks like github a...
1171,[Channel 34],1.744228e+09,Johnathan: Hey @[User Group 6]\n\nI have a pro...


In [305]:
df.iloc[0]

Channel ID                                           [Channel 45]
Thread ID                                       1714579526.341119
conversation    Collin: Hey @Calvin, we are migrating the call...
Name: 0, dtype: object

In [306]:
df.iloc[0]['conversation']

'Collin: Hey @Calvin, we are migrating the calling-auth service (which supposedly is not in use) to the new Argo deployment workflow. @Gabrielle is helping us with that. But since it is linked to a "legacy" hostname [URL] it needs some bigger effort if we want to do that.\n\nSince the service is supposed not be in use, and it was just used for experimenting, probably it makes sense to just move it to the new hostname and avoid any extra effort. But there is still some slight amount of traffic going through [URL].\n\n1. Do we know where the traffic is coming from?\n2. Do you know if it is doable to just change the hostname instead of using [URL] to use [URL] without a client change? I think I remember it being LP configurable?\n3. Or can we completely shutdown the traffic to ensure there is no disruption?\nCalvin: @Collin [URL] is the URL for the client to retrieve client config (SIP, TURN servers) through gRPC.  The feature is currently disabled.  It was previously enabled for a short 

In [307]:
for i in range(len(df))[:10]:
    print(df.iloc[i]['conversation'])


Collin: Hey @Calvin, we are migrating the calling-auth service (which supposedly is not in use) to the new Argo deployment workflow. @Gabrielle is helping us with that. But since it is linked to a "legacy" hostname [URL] it needs some bigger effort if we want to do that.

Since the service is supposed not be in use, and it was just used for experimenting, probably it makes sense to just move it to the new hostname and avoid any extra effort. But there is still some slight amount of traffic going through [URL].

1. Do we know where the traffic is coming from?
2. Do you know if it is doable to just change the hostname instead of using [URL] to use [URL] without a client change? I think I remember it being LP configurable?
3. Or can we completely shutdown the traffic to ensure there is no disruption?
Calvin: @Collin [URL] is the URL for the client to retrieve client config (SIP, TURN servers) through gRPC.  The feature is currently disabled.  It was previously enabled for a short time as 

In [303]:
# df.to_csv('/Users/anthony/Code/tn_slackbot/data/messages2024-04-03-2025-04-10_cleaned.csv', index=False)